In [1]:
#Exercise 1 to dobrze 

In [2]:
import pandas as pd

In [3]:
# Definiujemy potencjalne separatory i znaki dziesiętne
separatory = ['|', ';', ',']
znaki_dziesietne = ['.', ',']

# Próbujemy odczytać plik z różnymi separatorami i znakami dziesiętnymi
for separator in separatory:
    for znak_dziesietny in znaki_dziesietne:
        try:
            df = pd.read_csv("proj2_data.csv", sep=separator, decimal=znak_dziesietny)
            # Sprawdzamy, czy DataFrame ma przynajmniej jedną kolumnę z liczbami zmiennoprzecinkowymi
            if any(df[col].dtype == 'float64' for col in df.columns):
                # Zapisujemy DataFrame do pliku pickle
                df.to_pickle("proj2_ex01.pkl")
                print("DataFrame został pomyślnie wczytany i zapisany do proj2_ex01.pkl.")
                # Wyświetlamy pierwsze kilka wierszy DataFrame
                print(df.head())
                break
        except Exception as e:
            # Jeśli wystąpi błąd, próbujemy następnej kombinacji separatorów i znaków dziesiętnych
            continue
    else:
        continue
    break  # Wychodzimy z pętli zewnętrznej, jeśli sukces
else:
    print("Nie udało się wczytać DataFrame z pliku, używając żadnych zdefiniowanych separatorów i znaków dziesiętnych.")


DataFrame został pomyślnie wczytany i zapisany do proj2_ex01.pkl.
          full_name   field   language code  task_1  task_2  task_3  \
0  Rowan Harrington  drones     python  wej     3.1     2.0     4.4   
1        Nash Wyatt  racing       java  sfe     4.2     2.0     2.0   
2    Jadiel Ramirez   media  cplusplus  vaw     4.0     4.9     3.0   
3    Makaila Atkins  racing      swift  ugt     4.1     5.0     4.8   
4    Melanie Fuller  racing     python  owb     2.7     2.0     2.0   

   tasks_avg    task_grade jury_score  final_grade  
0   3.166667   dostateczny    3,5 pts  dostateczny  
1   2.733333  bardzo dobry        5 p       mierny  
2   3.966667         dobry        3.5       mierny  
3   4.633333         dobry          2  dostateczny  
4   2.233333  bardzo dobry      pts 2       mierny  


In [4]:
#Exercise 2 to dobrze 

In [5]:
# Wczytaj wartości skali z pliku tekstowego
with open("proj2_scale.txt", "r") as f:
    wartosci_skali = [line.strip() for line in f]

# Utwórz kopię początkowego DataFrame
df_kopia = df.copy()

# Iteruj przez kolumny DataFrame
for kolumna in df_kopia.columns:
    # Sprawdź, czy wartości w kolumnie są podzbiorem wartości skali
    if set(df_kopia[kolumna].unique()).issubset(set(wartosci_skali)):
        # Zastąp wartości ich odpowiednikami numerycznymi
        df_kopia[kolumna] = df_kopia[kolumna].apply(lambda x: wartosci_skali.index(x) + 1)

# Zapisz zmodyfikowany DataFrame do pliku pickle
df_kopia.to_pickle("proj2_ex02.pkl")

In [6]:
#Exercise 3 to dobrze

In [7]:
# Wczytaj wartości skali z pliku tekstowego
with open("proj2_scale.txt", "r") as f:
    wartosci_skali = [line.strip() for line in f]

# Utwórz kolejną kopię początkowego DataFrame
df_kopia2 = df.copy()

# Iteruj przez kolumny DataFrame
for kolumna in df_kopia2.columns:
    # Sprawdź, czy wartości w kolumnie są podzbiorem wartości skali
    if set(df_kopia2[kolumna].unique()).issubset(set(wartosci_skali)):
        # Zmień typ kolumny na kategoryczny
        df_kopia2[kolumna] = df_kopia2[kolumna].astype('category')
        # Ustaw kategorie dla kolumny tak, aby odzwierciedlały całą listę wczytaną z pliku tekstowego
        df_kopia2[kolumna] = pd.Categorical(df_kopia2[kolumna], categories=wartosci_skali, ordered=True)

# Zapisz zmodyfikowany DataFrame do pliku pickle
df_kopia2.to_pickle("proj2_ex03.pkl")

In [8]:
#Exercise 4 

In [9]:
import re

In [14]:

def extract_number(string):
    pattern = r"[-+]?\d*\.?\d+"
    match = re.search(pattern, string.replace(',', '.'))
    if match:
        return float(match.group())
    else:
        return None

df = pd.read_pickle("./proj2_ex03.pkl") # ./ wskazanie że plik znajduje się w bieżącym katalogu roboczym
extracted_numbers = {}

for column in df.select_dtypes(exclude=['number']).columns:
    nice_col = df[column].apply(lambda x: extract_number(str(x)))
    if any(nice_col.notna()):
        extracted_numbers[column] = nice_col

extracted_df = pd.DataFrame(extracted_numbers)

extracted_df.to_pickle("./proj2_ex04.pkl")
extracted_df

,jury_score
0,3.5
1,5.0
2,3.5
3,2.0
4,2.0
5,3.5
6,-1.0
7,0.0
8,NaN
9,NaN


In [15]:
#Exercise 5 

In [16]:

initial_df = pd.read_pickle("./proj2_ex01.pkl")

with open("./proj2_scale.txt", "r") as f:
    scale_values = [line.strip() for line in f]

encoded_dfs = []

for column in initial_df.select_dtypes(include=['object']):
    unique_values = initial_df[column].unique()

    if (len(unique_values) <= 10 and
        all(value.islower() for value in unique_values) and
        all(value not in scale_values for value in unique_values)):

        encoded_df = pd.get_dummies(initial_df[column])
        encoded_dfs.append(encoded_df)

for idx, encoded_df in enumerate(encoded_dfs, start=1):
    filename = f"./proj2_ex05_{idx}.pkl"
    encoded_df.to_pickle(filename)
    print(encoded_df)

    drones  media  racing  robotics
0     True  False   False     False
1    False  False    True     False
2    False   True   False     False
3    False  False    True     False
4    False  False    True     False
5     True  False   False     False
6    False   True   False     False
7    False  False   False      True
8    False   True   False     False
9     True  False   False     False
10    True  False   False     False
11   False  False    True     False
    cplusplus   java  python  swift
0       False  False    True  False
1       False   True   False  False
2        True  False   False  False
3       False  False   False   True
4       False  False    True  False
5       False  False    True  False
6       False  False   False   True
7        True  False   False  False
8       False   True   False  False
9       False   True   False  False
10      False  False   False   True
11      False  False    True  False
